Calibrate: MOT shutter delay, delay of OP shutters

# Creating new display

In [8]:
import matplotlib.pyplot as plt

In [1]:
timeline=tl.stack(
    init(t=-2),
    MOT(),
    MOT_detunedGrowth(),
    molasses(),
    OP(),
    magneticTrapping(),
    finish(MOT_ON=True),
)

variables = ["Anchor","shutter_MOT","AOM_MOT","shutter_repump","AOM_repump",
             "shutter_OP001","shutter_OP002","AOM_OP","coil_MOTlower__A","coil_MOTupper__A","lockbox_MOT__MHz"]

#tl.display(timeline)
def display_new(timeline, variables=None) :
    timeline = timeline.sort_values("time", ignore_index=True)
    
    max_time=timeline.loc[timeline["context"]!="ADwin_Finish","time"].max() # apart from the finish section
    
    timeline.loc[timeline["context"] == "ADwin_LowInit", "time"] = -0.5
    timeline.loc[timeline["context"] == "ADwin_Init",    "time"] = -0.25
    timeline.loc[timeline["context"] == "ADwin_Finish",  "time"] = max_time+0.25
    
    if variables is None : variables = timeline["variable"].unique()
    variables = sorted(variables, key=(lambda s: s[s.find("_") + 1]))
    
    analog_variables = { key: [s for s in variables if s.endswith(value)] for key, value in tl.ANALOG_SUFFIXES.items() }
    analog_variables = { key: value for key, value in analog_variables.items() if value }
    
    digital_variables = list( filter( lambda s : s not in [item for sublist in analog_variables.values() for item in sublist] and s!="Anchor" , variables ) )
    
    prop_cycle = plt.rcParams['axes.prop_cycle']
    colors = prop_cycle.by_key()['color']
    
    fig, axes = plt.subplots(
       len(analog_variables)+1 , sharex=True, figsize=(7.5, 7.5)
    )  # TODO: make this more flexible, preferably sth like %matplotlib
    
    fig.tight_layout()
    
    for key, axis in zip(analog_variables.keys(),axes[:-1]) :
        axis.set_ylabel(key+" [{}]".format(tl.ANALOG_SUFFIXES[key][2:]))
        for variable, color in zip(analog_variables[key],colors) :
            array=timeline.loc[ timeline["variable"] == variable, ["time", "value", "context"] ].to_numpy()
            axis.plot(array[:,0],array[:,1])
            axis.text(array[0,0],array[0,1],variable,color=color)
    
    axes[-1].set_ylabel("Digital channels")
    for variable, offset, color in zip(digital_variables,range(len(list(digital_variables))),colors) :
        baseline=offset/10.
        array=timeline.loc[ timeline["variable"] == variable, ["time", "value", "context"] ].to_numpy()
        axes[-1].step(array[:,0],array[:,1]+baseline,where="post",color=color)
        axes[-1].axhline(baseline, color=color, linestyle=":")
    axes[-1].set_yticks( [i/10. for i in range(len(list(digital_variables))) ] )
    axes[-1].set_yticklabels(digital_variables)
    # draw dotted lines of the same color at zero level, and on y axis make ticks+labels with the same color + name them with key
    # (the original ticks are not necessary)
    
    # shade init and finish:
    axes[-1].axvspan(-.75, 0, color='gray', alpha=0.3) 
    axes[-1].axvspan(max_time, max_time+.5, color='gray', alpha=0.3)
    
    anchors=timeline.loc[ timeline["variable"] == "Anchor", ["time", "context"] ].to_numpy()
    for anchorTime in anchors[:,0] :
        for axis in axes :
            axis.axvline(anchorTime, color='0.5', linestyle='--')
    
    ax2=axes[0].twiny()
    ax2.set_xlim(axes[0].get_xlim())
    ax2.set_xticks(list(anchors[:,0]))  # Set ticks at the specified x-values
    ax2.set_xticklabels(list(anchors[:,1]))
    
    def sync_axes(event):
        ax2.set_xlim(axes[0].get_xlim())
    
    # Connect the sync function to the 'xlim_changed' event
    axes[0].callbacks.connect('xlim_changed', sync_axes)
    
    # Display the plot
    plt.show()

NameError: name 'tl' is not defined

In [38]:
list(anchors[:,0])

[0.0, 15.0, 15.1, 15.105, 16.105]

In [17]:
axes[0].get_ylim()

(np.float64(-1.125), np.float64(1.625))

In [49]:
for t, c in timeline.loc[ timeline["variable"] == "Anchor", ["time", "context"] ].to_numpy() : #[:,0]
    print(t,c)

0.0 InitialAnchor
15.0 MOT
15.1 MOT
15.105 molasses
16.105 finalRamps


In [50]:
analog_variables

{'Current': ['coil_MOTlowerPlus__A',
  'coil_compensationX__A',
  'coil_compensationY__A',
  'coil_MOTupperPlus__A',
  'coil_MOTlower__A',
  'coil_MOTupper__A'],
 'Frequency': ['lockbox_MOT__MHz']}

In [32]:
timeline.loc[ timeline["variable"] == "coil_MOTlower__A", ["time", "value", "context"] ].to_numpy()[:,1]

array([-1.0, -1.0, -0.999828, ..., -0.99997, -0.999985, -1.0],
      dtype=object)

# Tests

In [4]:
timeline=tl.stack(
    init(t=-2),
    MOT(),
    MOT_detunedGrowth(),
    molasses(),
    finish(),
)

#tl.display(timeline)

In [ ]:
timeline=tl.stack(
    init(t=-2#,shutter_imaging=0,AOM_imaging=1,trigger_camera=0
        ),
    MOT(t=0.0),
    MOT_detunedGrowth(),
    molasses(),
    OP()
)

tl.display(timeline,variables=["Anchor","shutter_MOT","AOM_MOT","shutter_repump","AOM_repump",
                               "shutter_OP001","shutter_OP002","AOM_OP","coil_MOTlower__A","coil_MOTupper__A","lockbox_MOT__MHz"])

# Imports, connections, devices, constants

In [1]:
import sys
sys.path.append("..")

import importlib.resources as resources
import pathlib as pl

import pandas as pd
import numpy as np
from munch import Munch
from wigner_time import connection as con
from wigner_time import timeline as tl
from wigner_time import adwin as adwin
from wigner_time import constructor as construct
import wigner_time

from copy import deepcopy

%matplotlib qt

In [2]:
connections = con.connection(
    ["shutter_MOT", 1, 11],
    ["shutter_repump", 1, 12],
    ["shutter_OP001", 1, 14],
    ["shutter_OP002", 1, 15],
    ["AOM_MOT", 1, 1],
    ["AOM_repump", 1, 2],
    ["AOM_OP_aux", 1, 30],  # should be set to 0 always
    ["AOM_OP", 1, 31],
    ["coil_compensationX__A", 4, 7],
    ["coil_compensationY__A", 3, 2],
    ["coil_MOTlower__A", 4, 1],
    ["coil_MOTupper__A", 4, 3],
    ["coil_MOTlowerPlus__A", 4, 2],
    ["coil_MOTupperPlus__A", 4, 4],
    ["lockbox_MOT__MHz", 3, 8],
)

devices = pd.DataFrame(
    columns=["variable", "unit_range", "safety_range"],
    data=[
        ["coil_compensationX__A", (-3, 3), (-3, 3)],
        ["coil_compensationY__A", (-3, 3), (-3, 3)],
        ["coil_MOTlower__A", (-5, 5), (-5, 5)],
        ["coil_MOTupper__A", (-5, 5), (-5, 5)],
        ["coil_MOTlowerPlus__A", (-5, 5), (-5, 5)],
        ["coil_MOTupperPlus__A", (-5, 5), (-5, 5)],
        #["lockbox_MOT__V", (-10, 10)],
        ["lockbox_MOT__MHz", (-200, 200)],
    ],
)

# I dislike global variables but, unfortunately, a reference will probably still need to be available in the same namespace as these functions for convenience.
constants = Munch(
    safety_factor=1.1,
#    factor__VpMHz=0.05,
    lag_MOTshutter=2.48e-3,
    Compensation=Munch(
        Z__A=-0.1,
        Y__A=1.5,
        X__A=0.25,
    ),
    OP=Munch(
        lag_AOM_on=15e-6,
        lag_shutter_on=1.5e-3,
        lag_shutter_off=1.5e-3,
        duration_shutter_on=140e-6,
        duration_shutter_off=600e-6,
    ),
    AI=Munch(
        lag_shutter_on=2.2e-3,
        lag_shutter_off=1.9e-3,
    )
)


# Fundamental sample preparation functions

## Magnetic trapping

In [60]:
def OP(durationExposition=80e-6, durationCoilRamp=50e-6, i=-0.12, pt=3, **kwargs) :
    fullDuration=durationExposition+durationCoilRamp
    return tl.stack(
        tl.ramp(
            coil_MOTlower__A= i,
            coil_MOTupper__A=-i,
            duration=durationCoilRamp,
            fargs={"ti": pt},
            context="OP",**kwargs
        ),
        tl.set("AOM_OP",[[-0.1,0],[durationCoilRamp,1],[fullDuration,0]]),
        tl.set("shutter_OP001",[[durationCoilRamp-constants.OP.lag_shutter_on,1],[0.1,0]]),
        tl.set("shutter_OP002",[[fullDuration-constants.OP.lag_shutter_off,0],[0.1,1]]),
        tl.set(AOM_repump=0,shutter_repump=0,t=fullDuration),
        tl.anchor(fullDuration,context="OP")
        #     # I switch off the AOM close before opening the first shutter, but safely enough, not to flash too soon
        #     [
        #         "shutter_OP001",
        #         [ time - constants.OP.lag_aom_on - constants.safety_factor * (constants.OP.lag_shutter_on + constants.OP.duration_shutter_on), 1 ],
        #     ],
        #     [
        #         "AOM_OP",
        #         [time - constants.OP.lag_aom_on + 9.5e-6, 1], #TODO: what is this constant?????
        #     ],
        #     [
        #         "shutter_OP002",
        #         [ time + duration_exposure - (2 - constants.safety_factor) * constants.OP.lag_shutter_off, 0 ],
        #     ],
        #     ["AOM_OP", [ time+duration_exposure + 0.5e-6, 0 ] ], # TODO: another mysterious constant
        #     ["AOM_repump", [ time+duration_exposure, 0 ] ],
        #     ["shutter_repump", [ time+duration_exposure, 0 ] ],
        #     # Shutters switched back, to reinitialize them before any additional optical pumpings later.
        #     ["shutter_OP001", [ time+duration_exposure, 0 ] ],
        #     ["shutter_OP002", [ time+duration_exposure + constants.safety_factor * (constants.OP.lag_shutter_on+constants.OP.duration_shutter_on), 1 ] ],
        #     context=context,
        #     timeline=tline,
        )


def pull_coils(duration, l, u, lp=-constants.Compensation.Z__A, up=constants.Compensation.Z__A, pt = 3, **kwargs) :
    return tl.ramp(
        coil_MOTlower__A=l,
        coil_MOTupper__A=u,
        coil_MOTlowerPlus__A=lp,
        coil_MOTupperPlus__A=up,
        fargs={"ti": pt},
        duration=duration,
        **kwargs,
    )

def magneticTrapping(durationInitial=50e-6, li=-1.8, ui=-1.7, durationStrengthen=3e-3, ls=-4.8, us=-4.7, **kwargs) :
    return tl.stack(
        pull_coils(durationInitial,li,ui,context="magneticTrapping",**kwargs),
        pull_coils(durationStrengthen,ls,us,t=durationInitial),
        tl.anchor(durationInitial+durationStrengthen,context="magneticTrapping")
    )

## Cloud collection & cooling

In [ ]:
def init(**kwargs) :
    """
    Creates an experimental timeline for the initialization of every device.
    """
    return tl.stack(
        tl.create(
            lockbox_MOT__MHz=0.0,
            coil_compensationX__A= constants.Compensation.X__A,
            coil_compensationY__A= constants.Compensation.Y__A,
            coil_MOTlowerPlus__A = -constants.Compensation.Z__A,
            coil_MOTupperPlus__A =  constants.Compensation.Z__A,
            AOM_MOT = 1,
            AOM_repump = 1,
            AOM_OP_aux = 0, # TODO: USB-controlled AOMs should be treated on a higher level
            AOM_OP = 1,
            shutter_MOT = 0,
            shutter_repump = 0,
            shutter_OP001 = 0,
            shutter_OP002 = 1,
            context="ADwin_LowInit",
            **kwargs
        ),
        tl.anchor(t=0.0,relative={"time" : False},context="InitialAnchor")
    )


def finish(wait=1, lA=-1., uA=-0.98, MOT_ON=True, **kwargs) :
    duration=1e-2
    return tl.stack(
        tl.anchor(wait,context="finalRamps"),
        tl.ramp(
            lockbox_MOT__MHz=0.0,
            coil_MOTlower__A = lA,
            coil_MOTupper__A = uA,
            coil_compensationX__A= constants.Compensation.X__A,
            coil_compensationY__A= constants.Compensation.Y__A,
            coil_MOTlowerPlus__A = -constants.Compensation.Z__A,
            coil_MOTupperPlus__A =  constants.Compensation.Z__A,
            duration=duration,
            context="finalRamps"
        ),
        tl.set(
            AOM_MOT = 1,
            AOM_repump = 1,
            AOM_OP_aux = 0, # TODO: USB-controlled AOMs should be treated on a higher level
            AOM_OP = 1,
            shutter_MOT = int(MOT_ON),
            shutter_repump = int(MOT_ON),
            shutter_OP001 = 0,
            shutter_OP002 = 1,
            t=0.1,
            context="ADwin_Finish",
            **kwargs
        )
    )


def MOT(duration=15, lA=-1., uA=-0.98, **kwargs) :
    return tl.stack(
        tl.set(
#           waitChannel = 0,
            shutter_MOT=1,
            shutter_repump=1,
            coil_MOTlower__A=lA,
            coil_MOTupper__A=uA,
            context='MOT',
            **kwargs
        ),
        tl.anchor(duration)
    )


def MOT_detunedGrowth(duration=100e-3, durationRamp=10e-3, toMHz=-5, pt=3, **kwargs) :
    return tl.stack(
        tl.ramp(
            lockbox_MOT__MHz=toMHz,
            duration=durationRamp,
            fargs={"ti": pt},
            context='MOT',
            **kwargs
        ),
        tl.anchor(duration)
    )


def molasses(duration=5e-3, durationCoilRamp=9e-4, durationLockboxRamp=1e-3, toMHz=-90, coil_pt=3, lockbox_pt=3, **kwargs) :

    return tl.stack(
        tl.ramp(
            coil_MOTlower__A=0,
            coil_MOTupper__A=0, # TODO: can these be other than 0 (e.g. for more perfect compensaton?)
            duration=durationCoilRamp,
            fargs={"ti": coil_pt},
            context="molasses",
            **kwargs
        ),
        tl.ramp(
            lockbox_MOT__MHz=toMHz,
            duration=durationLockboxRamp,
            fargs={"ti": lockbox_pt},
        ),
        tl.set(
            shutter_MOT=[duration - constants.lag_MOTshutter,0],
            AOM_MOT=[duration,0]
        ),
        tl.anchor(duration,context="molasses")
        
    )

# Absorption imaging

In [ ]:
connections=pd.concat([connections,con.connection(["shutter_imaging", 1, 13],["AOM_imaging", 1, 5],["trigger_camera", 1, 0])])

In [ ]:
def trigger_camera(time, exposure, **kwargs) :
    return tl.set(
        ["trigger_camera",[time,1]],
        ["trigger_camera",[time+exposure,0]],
        **kwargs)

def flash_light(time, exposure, **kwargs) :
    sf = constants.safety_factor
    return tl.stack(
        tl.set(
            ["AOM_imaging",[time,1]],
            ["AOM_imaging",[time+exposure,0]],
            **kwargs),
        tl.set(
            ["shutter_imaging",[time-exposure*(sf-1)-constants.AI.lag_shutter_on,1]],
            ["shutter_imaging",[time+exposure*sf,0]]), # TODO: what to write here exactly?
    )

def expose_camera(time, exposure, **kwargs) :
    sf = constants.safety_factor
    return tl.stack(
        flash_light(time,exposure,**kwargs),
        trigger_camera(time-exposure*(sf-1)/2, exposure*sf)
    )

def take_image_plus_Bg(time, exposure, delayBg, **kwargs) :
    return tl.stack(
        expose_camera(time, exposure, **kwargs), # taking At image
        trigger_camera(time+delayBg, exposure*constants.safety_factor), # taking Bg_At image
    )    

def imaging_absorption(time, exposure, delayBg, delayLi, exposureBlow=None, **kwargs) :
    """
    Creates an experimental timeline for absorption imaging.
    """
    context = "imaging_absorption"
    return tl.stack(
        tl.set(AOM_imaging=[time-constants.safety_factor*exposure,0],context=context,**kwargs), # initializing the AOM
        take_image_plus_Bg(time, exposure, delayBg), # taking At + Bg_At image
        flash_light(time+5*delayBg,exposureBlow) if exposureBlow is not None else None, # blow out the atoms in between
        take_image_plus_Bg(time+delayLi, exposure, delayBg), # taking Li + Bg_Li image
    )

# Tests old

## Testing the difference between `globalRelative=True/False` in `next`

In [ ]:
timeline=tl.stack(
    init(t=-2,coil_MOTlower__A=.0,coil_MOTupper__A=.0),
    tl.next(coil_MOTlower__A=-3.0, t=1., time_start=0., context="any"),
    tl.next(coil_MOTupper__A=-4.0, t=1.5, time_start=0.5),
    tl.next(lockbox_MOT__MHz=-5, t=1.5, globalRelative=False)
)

In [ ]:
tl.display(timeline)#,variables=["coil_MOTlower__A","coil_MOTupper","lockbox_MOT__MHz"])

In [ ]:
timeline=tl.stack(
    init(t=-2#,shutter_imaging=0,AOM_imaging=1,trigger_camera=0
        ),
    MOT(t=0.0),
    MOT_detunedGrowth(),
    tl.wait(1),
    tl.next(
        lockbox_MOT__MHz=0,
        coil_MOTlower__A=-1.0,
        coil_MOTupper__A=-0.98,
        t=.1,
        fargs={"ti": 3},context='sanitize'
    ),
    tl.set(AOM_MOT=1,AOM_repump=1,AOM_imaging=1)
)

In [ ]:
prev=tl.previous(timeline,"coil_MOTlower__A")
prev.values[0]=1#["value"]
prev

# Thomas’s solution

In [ ]:
defaults = Munch()

# NOTE: MOTplus coils are part of the compensation and so should default to the compensation values.
defaults.MOT = Munch(
    lockbox_MOT__V=0.0,
    shutter_MOT=1, # TODO: why the shutter values here?
    shutter_repump=1,
    coil_MOTlower__A=-1.0,
    coil_MOTupper__A=-0.98,
)

defaults.molasses = Munch(
    duration_cooling=5e-3,
    duration_ramp=1e-3,
    shift__MHz=-80,
    fraction_ramp_duration=0.9,
    coil_MOTlower__A=0.0,
    coil_MOTupper__A=0.0,
)

defaults.magnetic = Munch(
    delay=1e-3,
    quadrupole=Munch(duration_ramp=50e-6, coil_MOTlower__A=-1.8, coil_MOTupper__A=-1.7),
    strong=Munch(duration_ramp=3e-3, coil_MOTlower__A=-4.8, coil_MOTupper__A=-4.7),
)

defaults.finish = Munch(set={}, ramp={})
defaults.finish.set = Munch(
    AOM_MOT=1,
    AOM_repump=1,
    AOM_repump__V=5,
    AOM_imaging=1,
    AOM_OP_aux=0,
    AOM_OP=1,
    AOM_science=1,
    AOM_science__V=0.0,
    AOM_ref=1,
    shutter_MOT=1,
    shutter_repump=1,
    shutter_imaging=0,
    shutter_OP001=0,
    shutter_OP002=1,
    shutter_science=1,
    shutter_transversePump=0,
    shutter_coupling=0,
    trigger_TC__V=0,
    photodiode__V=0,
)
defaults.finish.ramp = Munch(
    coil_MOTlower__A=0.0,
    coil_MOTupper__A=0.0,
    coil_MOTlowerPlus__A=-constants.Bfield_compensation_Z__A,
    coil_MOTupperPlus__A=constants.Bfield_compensation_Z__A,
    coil_compensationX__A=constants.Bfield_compensation_X__A,
    coil_compensationY__A=constants.Bfield_compensation_Y__A,
    lockbox_MOT__V=lock_box.to_V(0.0),
)

In [ ]:
def init():
    """
    Creates an experimental timeline for the initialization of every relevant variable.
    """
    return tl.create(
        lockbox_MOT__V=0.0,
        coil_compensationX__A=constants.Bfield_compensation_X__A,
        coil_compensationY__A=constants.Bfield_compensation_Y__A,
        coil_MOTlowerPlus__A=-constants.Bfield_compensation_Z__A,
        coil_MOTupperPlus__A=constants.Bfield_compensation_Z__A,
        AOM_MOT=1,
        AOM_repump=1,
        AOM_OP_aux=0,
        AOM_OP=1,
        shutter_MOT=0,
        shutter_repump=0,
        shutter_OP001=0,
        shutter_OP002=1,
        context="ADwin_LowInit",
    )

def MOT(
    detuning__MHz=-5,
    duration=10.0,
    durationFinal=1.0,
    # ===
    time_start=None,
    variables: Munch | None = None,
    variables_default=defaults.MOT,
    timeline=init(),
    context="MOT",
):
    """
    Creates a Magneto-Optical Trap.

    """
    _time_start, _variables = construct.time_and_arguments(
        time_start, variables, variables_default, timeline
    )

    return tl.update(
        tl.create(
            shutter_MOT=_variables.shutter_MOT,
            shutter_repump=_variables.shutter_repump,
            coil_MOTlower__A=_variables.coil_MOTlower__A,
            coil_MOTupper__A=_variables.coil_MOTupper__A,
            # ===
            timeline=timeline,
            context=context,
        ),
        tl.next(
            lockbox_MOT__V=lock_box.to_V(detuning__MHz), t=durationFinal, time_start=duration-durationFinal, context="MOT_grow"
        ),
    )


def molasses(
    duration_cooling=5e-3,
    duration_ramp=1e-3,
    detuning__MHz=-80,
    # ===
    # Specific values above ↑
    # Repeated values below ↓
    # ===
    time_start=None,
    variables: Munch | None = None,
    variables_default=defaults.molasses,
    timeline=MOT(),
    context="molasses",
):
    _time_start, _variables = construct.time_and_arguments(
        time_start, variables, variables_default, timeline
    )

    if duration_ramp >= duration_cooling:
        raise ValueError("duration_ramp should be smaller than duration_cooling!")

    return tl.update(
        tl.create(
            AOM_MOT=[_time_start + duration_cooling, 0],
            shutter_MOT=[_time_start + duration_cooling - constants.lag_MOTshutter, 0],
            timeline=timeline,
            context=context,
        ),
        tl.next(
            coil_MOTlower__A=_variables.coil_MOTlower__A,
            coil_MOTupper__A=_variables.coil_MOTupper__A,
            t=duration_ramp,
        ),
        tl.next(lockbox_MOT__V=lock_box.to_V(detuning__MHz), t=duration_ramp),
    )

In [ ]:
init()

In [ ]:
tl.display(molasses(timeline=MOT(variables={"coil_MOTlower__A":-.5})))

# Params layer

In [ ]:
params_default = Munch(
    MOT=Munch(
        duration_grow=15.0,
        duration_final=100e-3, # should be smaller than duration_grow
        shift__MHz=-5.0,
        duration_ramp=10e-3,
        coil_lower__A=-1,
        coil_upper__A=-0.98, # TODO why the asymmetry? this was probably part of magnetic compensation
    ),
    molasses=Munch(
        duration_cooling=5e-3,
        duration_ramp=1e-3, # should be smaller than duration_cooling
        shift__MHz=-80,
        fraction_ramp_duration=0.9,
        ),
    OP=Munch(
        duration_exposure=80e-6,
        duration_ramp_coils_dipole=500e-6,
        homogeneous_field__A=-0.12,
    ),
    trapping_magnetic=Munch(
        delay_magnetic_trap=1e-3,
        duration_ramp_coils_quadrupole=50e-6,
        duration_ramp_coils_strengthen=3e-3,
        lowerMOT_quadrupole__A=-1.8,
        upperMOT_quadrupole__A=-1.7, # TODO why the asymmetry? this was probably part of magnetic compensation
        lowerMOT_strengthen__A=-4.8,
        upperMOT_strengthen__A=-4.7, # TODO why the asymmetry? this was probably part of magnetic compensation
    ),
    transport_magnetic=Munch(
        delay=5e-3,
        duration=150e-3,
        lowerMOT__A=-0.3121632,
        upperMOT__A=-4.748368,
        lowerMOTPlus__A=-4.648368,
        upperMOTPlus__A=-4.748368,
        parameter_tanh=2.25,
    ),
    finish=Munch(
        duration_until_finish = 0,
        duration_ramp = 1e-3,
        lockbox_initial__V = 0,
    )
)

In [ ]:
def init():
    return preparation_functions.init()

def MOT(tline, time, params,context=""):
    tline = preparation_functions.MOT(tline, time, params.MOT.lowerMOT__A, params.MOT.upperMOT__A)
    tline = preparation_functions.MOT_detunedGrowth(tline, time+params.MOT.duration_grow-params.MOT.duration_final, params.MOT.duration_ramp, 
                                             params.MOT.shift__MHz,pt=3)
    return tline, time+params.MOT.duration_grow

def molasses(tline, time, params):
    tline = preparation_functions.molasses(tline, time, params.molasses.duration_cooling, params.molasses.duration_ramp, params.molasses.duration_ramp, 
                                            params.molasses.shift__MHz)
    return tline, time+params.molasses.duration_cooling

def OP(tline, time, params):
    #def OP(tline, time, duration_exposure, duration_ramp_coils_dipole, duration_shutter_on, homogenous_field__A,coil_parameter_tanh=3):
    tline = preparation_functions.OP(tline, time, params.OP.duration_exposure, params.OP.duration_ramp_coils_dipole, params.OP.homogenous_field__A)
    return tline, time+params.OP.duration_exposure

def magnetic_field(tline, time, params):
    tline = preparation_functions.pull_coils(tline, time, params.trapping_magnetic.duration_ramp_coils_quadrupole, params.trapping_magnetic.lowerMOT_quadrupole__A,
                                                    params.trapping_magnetic.upperMOT_quadrupole__A,context="trapping_magnetic")
    time = time+params.trapping_magnetic.duration_ramp_coils_quadrupole
    tline = preparation_functions.pull_coils(tline, time, params.trapping_magnetic.duration_ramp_coils_strengthen, params.trapping_magnetic.lowerMOT_strengthen__A,
                                             params.trapping_magnetic.upperMOT_strengthen__A,context="trapping_magnetic")
    return tline, time+params.trapping_magnetic.duration_ramp_coils_strengthen

def transport_magnetic(tline, time, params):
    tline = preparation_functions.pull_coils(tline, time, params.transport_magnetic.duration, params.transport_magnetic.lowerMOT__A,
                                            params.transport_magnetic.upperMOT__A,params.transport_magnetic.lowerMOTPlus__A,params.transport_magnetic.upperMOTPlus__A,
                                            context = "magnetic_transport")
    return tline, time+params.transport_magnetic.duration

def switch_off_trap_magnetic(tline, time, params):
    tline = preparation_functions.pull_coils(tline, time, params.switch_off_trap_magnetic.duration, 0,0)
    return tline, time+params.switch_off_trap_magnetic.duration

#parameter??
def transport_back_magnetic(tline, time, params):
    tline = preparation_functions.transport_back_magnetic(tline, time, params.transport_back_magnetic.duration, params.transport_back_magnetic.lowerMOT_strengthen__A, params.transport_back_magnetic.upperMOT_strengthen__A)
    return tline, time+params.transport_back_magnetic.duration

def finish(tline, time, params):
    return preparation_functions.finish(tline, time, params.finish.duration_ramp, params.finish.lockbox_initial__V, params.finish.science_AOM__V, params.finish.coil_lowerMOT__A, params.finish.coil_upperMOT__A)

In [ ]:
def prepare_atoms(params):
    tline = init()
    tline,timeMOT = MOT(tline, 0, params)
    tline,timemolasses = molasses( *MOT(tline, 0, params), params)
    tline,timeOP = OP(tline, timemolasses, params)
    tline,timeTrapping = magnetic_field(tline, timeOP+params.trapping_magnetic.delay_magnetic_trap, params)
    tline = finish(tline, timeTrapping, params)
    return tline

In [ ]:
params=deepcopy(params_default)
params.finish.duration_until_finish = 500e-3

tline = prepare_atoms(params)

In [ ]:
%matplotlib qt
tl.display(tline,variables=['lockbox_MOT__V','AOM_MOT','AOM_repump','shutter_repump','AOM_repump__V'],xlim=[15,15.05])